In [1]:
# Celda 1: Imports básicos y configuración
import sys
import os
import pandas as pd
import numpy as np
from pathlib import Path
import sqlite3
import time
from datetime import datetime
import re

# Configuración de paths
project_root = Path.cwd().parent.parent
sys.path.append(str(project_root))

# Importar configuración
from config import APIFY_API_KEY, DB_PATH
from src.utils.database import get_db_connection, insert_dataframe_to_table

# Importar ApifyClient
from apify_client import ApifyClient

# Configuración de Apify
client = ApifyClient(APIFY_API_KEY)

# Para análisis de texto (opcional, si no tienes textblob instalado)
# !pip install textblob
# from textblob import TextBlob
# from textblob.exceptions import TranslatorError

In [2]:
# Celda 2: Funciones de la base de datos para Reviews
def create_reviews_table():
    """
    Crea la tabla Reviews si no existe
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    
    create_table_sql = """
    CREATE TABLE IF NOT EXISTS Reviews (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        tripadvisor_id INTEGER,
        restaurante_id INTEGER,
        titulo TEXT,
        contenido TEXT,
        rating INTEGER,
        fecha TEXT,
        usuario TEXT,
        ubicacion_usuario TEXT,
        sentimiento TEXT,
        temas TEXT,
        fecha_insercion TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (restaurante_id) REFERENCES Restaurantes(id)
    );
    """
    
    try:
        cursor.execute(create_table_sql)
        conn.commit()
        print("✅ Tabla 'Reviews' creada/verificada")
    except Exception as e:
        print(f"❌ Error creando tabla Reviews: {e}")
    finally:
        conn.close()

def get_restaurantes_sin_reviews():
    """
    Retorna los restaurantes que no tienen reviews en la tabla Reviews
    """
    conn = get_db_connection()
    
    query = """
    SELECT r.id, r.tripadvisor_id, r.nombre 
    FROM Restaurantes r
    LEFT JOIN Reviews rev ON r.id = rev.restaurante_id
    WHERE rev.id IS NULL
    GROUP BY r.id
    HAVING COUNT(rev.id) = 0
    """
    
    try:
        df_restaurantes = pd.read_sql_query(query, conn)
        print(f"📊 Restaurantes sin reviews: {len(df_restaurantes)}")
        return df_restaurantes
    except Exception as e:
        print(f"❌ Error obteniendo restaurantes sin reviews: {e}")
        return pd.DataFrame()
    finally:
        conn.close()

def guardar_reviews_en_bd(df_reviews):
    """
    Guarda las reviews procesadas en la base de datos
    """
    if not df_reviews.empty:
        try:
            conn = get_db_connection()
            df_reviews.to_sql('Reviews', conn, if_exists='append', index=False)
            conn.close()
            print(f"✅ {len(df_reviews)} reviews guardadas en la BD")
        except Exception as e:
            print(f"❌ Error guardando reviews: {e}")
    else:
        print("📭 No hay reviews para guardar")

In [3]:
# Celda 3: Funciones de scraping de reviews
def get_reviews_por_tripadvisor_id(tripadvisor_id, restaurante_id, nombre_restaurante, max_reviews=20):
    """
    Obtiene reviews para un restaurante específico usando su tripadvisor_id
    """
    print(f"📖 Obteniendo reviews para: {nombre_restaurante} (ID: {tripadvisor_id})")
    
    reviews_data = []
    
    # Configurar input para Apify - URL específica del restaurante
    restaurant_url = f"https://www.tripadvisor.es/Restaurant_Review-g187499-d{tripadvisor_id}"
    
    run_input = {
        "startUrls": [{"url": restaurant_url}],
        "maxItems": max_reviews,  # Límite prudente
        "reviewsLanguage": "es",
        "proxyConfiguration": {"useApifyProxy": True},
        "includeReviews": True
    }
    
    try:
        # Ejecutar scraper de reviews
        run = client.actor("lazargjorgjevski/tripadvisor-scraper").call(run_input=run_input)
        
        # Procesar reviews
        for item in client.dataset(run["defaultDatasetId"]).iterate_items():
            if 'reviews' in item:
                for review in item['reviews']:
                    review_data = {
                        'tripadvisor_id': tripadvisor_id,
                        'restaurante_id': restaurante_id,
                        'titulo': review.get('title', ''),
                        'contenido': review.get('text', ''),
                        'rating': review.get('rating', 0),
                        'fecha': review.get('date', ''),
                        'usuario': review.get('user', {}).get('name', ''),
                        'ubicacion_usuario': review.get('user', {}).get('location', ''),
                        'sentimiento': '',  # Se calculará después
                        'temas': ''         # Se extraerá después
                    }
                    reviews_data.append(review_data)
        
        print(f"✅ {len(reviews_data)} reviews obtenidas para {nombre_restaurante}")
        
    except Exception as e:
        print(f"❌ Error obteniendo reviews para {nombre_restaurante}: {e}")
    
    return reviews_data

In [4]:
# Celda 4: Funciones de procesamiento de texto (análisis de sentimiento y temas)
def analizar_sentimiento_espanol(texto):
    """
    Analiza sentimiento para texto en español (sin dependencias externas)
    """
    if not texto or not isinstance(texto, str):
        return 0
    
    texto_lower = texto.lower()
    
    # Diccionario de palabras con pesos sentimentales
    palabras_sentimiento = {
        'excelente': 0.8, 'fantástico': 0.7, 'maravilloso': 0.7, 'perfecto': 0.8,
        'delicioso': 0.6, 'increíble': 0.6, 'genial': 0.6, 'recomiendo': 0.5,
        'bueno': 0.4, 'agradable': 0.3, 'correcto': 0.2, 'aceptable': 0.1,
        
        'terrible': -0.8, 'horrible': -0.8, 'decepcionante': -0.7, 'pésimo': -0.8,
        'malo': -0.6, 'fatal': -0.7, 'asqueroso': -0.8, 'evitar': -0.7,
        'lento': -0.4, 'caro': -0.3, 'sucio': -0.6, 'desastre': -0.7
    }
    
    # Calcular score basado en palabras clave
    score = 0
    count = 0
    
    for palabra, peso in palabras_sentimiento.items():
        if palabra in texto_lower:
            score += peso
            count += 1
    
    # Normalizar score si encontramos palabras
    if count > 0:
        return score / count
    else:
        return 0

def extraer_temas(texto):
    """
    Extrae temas clave de las reviews
    """
    if not texto or not isinstance(texto, str):
        return ''
    
    texto_lower = texto.lower()
    temas = []
    
    # Palabras clave por categorías
    categorias = {
        'comida': ['comida', 'plato', 'sabor', 'menu', 'carta', 'cocina', 'postre'],
        'servicio': ['servicio', 'camarero', 'mesero', 'atencion', 'trato', 'amable'],
        'precio': ['precio', 'caro', 'barato', 'valor', 'cuenta', 'pago'],
        'ambiente': ['ambiente', 'decoracion', 'musica', 'limpieza', 'local', 'terraza'],
        'tiempo': ['espera', 'reserva', 'esperar', 'demora', 'rapido', 'lento']
    }
    
    for categoria, palabras in categorias.items():
        for palabra in palabras:
            if palabra in texto_lower and categoria not in temas:
                temas.append(categoria)
                break
    
    return ', '.join(temas)

def procesar_reviews(df_reviews):
    """
    Procesa las reviews: sentimiento y temas
    """
    df = df_reviews.copy()
    
    # Analizar sentimiento
    df['sentimiento_score'] = df['contenido'].apply(analizar_sentimiento_espanol)
    df['sentimiento'] = df['sentimiento_score'].apply(
        lambda x: 'negativo' if x < -0.1 else 'positivo' if x > 0.1 else 'neutral'
    )
    
    # Extraer temas
    df['temas'] = df['contenido'].apply(extraer_temas)
    
    return df

In [5]:
# Celda 5: Función principal de ejecución
def obtener_reviews_incremental(max_reviews_por_restaurante=15, delay=2):
    """
    Obtiene reviews solo para restaurantes que no las tienen
    """
    print("🎯 Iniciando scraping incremental de reviews...")
    print("=" * 50)
    
    # 1. Crear tabla si no existe
    create_reviews_table()
    
    # 2. Obtener restaurantes sin reviews
    df_restaurantes = get_restaurantes_sin_reviews()
    
    if df_restaurantes.empty:
        print("🎉 Todos los restaurantes ya tienen reviews!")
        return pd.DataFrame()
    
    print(f"🔍 Encontrados {len(df_restaurantes)} restaurantes sin reviews")
    all_reviews = []
    
    # 3. Iterar por cada restaurante sin reviews
    for i, (_, restaurante) in enumerate(df_restaurantes.iterrows(), 1):
        print(f"\n[{i}/{len(df_restaurantes)}] ", end="")
        
        reviews = get_reviews_por_tripadvisor_id(
            restaurante['tripadvisor_id'],
            restaurante['id'],
            restaurante['nombre'],
            max_reviews_por_restaurante
        )
        
        if reviews:
            all_reviews.extend(reviews)
        
        # Pausa para no saturar
        if i < len(df_restaurantes):
            print(f"⏰ Esperando {delay} segundos...")
            time.sleep(delay)
    
    # 4. Procesar y guardar reviews
    if all_reviews:
        df_reviews = pd.DataFrame(all_reviews)
        
        # Analizar sentimiento y temas
        df_reviews = procesar_reviews(df_reviews)
        
        # Guardar en BD
        guardar_reviews_en_bd(df_reviews)
        
        print(f"\n🎯 Proceso completado. {len(df_reviews)} reviews guardadas")
        return df_reviews
    else:
        print("📭 No se obtuvieron reviews nuevas")
        return pd.DataFrame()

In [6]:
# Celda 6: Ejecución y verificación
# Ejecutar el scraping incremental
df_reviews_obtenidas = obtener_reviews_incremental(max_reviews_por_restaurante=10, delay=3)

# Verificar resultados
if not df_reviews_obtenidas.empty:
    print("\n📊 Resumen de reviews obtenidas:")
    print(f"Total reviews: {len(df_reviews_obtenidas)}")
    print(f"Distribución de sentimientos:")
    print(df_reviews_obtenidas['sentimiento'].value_counts())
    
    # Mostrar preview
    print("\n👀 Preview de las reviews:")
    print(df_reviews_obtenidas[['titulo', 'rating', 'sentimiento', 'temas']].head())
else:
    print("No se obtuvieron nuevas reviews")

🎯 Iniciando scraping incremental de reviews...
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
✅ Tabla 'Reviews' creada/verificada
📍 Conectando a: /Users/administrator/MASTER_Data_Science/TFM/tfme-horeca-intelligence/tfm_database.db
📍 ¿Existe el archivo? True
✅ Conexión a la BD SQLite exitosa.
📊 Restaurantes sin reviews: 40
🔍 Encontrados 40 restaurantes sin reviews

[1/40] 📖 Obteniendo reviews para: Restaurante Chino Gran Muralla 2 (ID: 5449694)
❌ Error obteniendo reviews para Restaurante Chino Gran Muralla 2: Actor with this name was not found
⏰ Esperando 3 segundos...

[2/40] 📖 Obteniendo reviews para: Jardins Gala (ID: 27533952)
❌ Error obteniendo reviews para Jardins Gala: Actor with this name was not found
⏰ Esperando 3 segundos...

[3/40] 📖 Obteniendo reviews para: Terram (ID: 23193586)
❌ Error obteniendo reviews para Terram: Actor with this name was not found
⏰ Esp

KeyboardInterrupt: 